In [5]:
import numpy as np
import math
from numpy.random import randint, choice

In [6]:
def init_population(pop_size, chromosome_length):
    """
    init_population(pop_size, chromosome_length)
    function that initializes a binary population to be used in a discrete
    genetic algorithm. The function creates 'pop_size' random binary bits (genes) 
    of the chromosome of length 'chromosome_length'
    po_size: the size of the binary population being created
    chromosome_length: the number of bits per individual/chromosome
    returns: the binary population
    """
    # Generate random values between 0 and 1. Generate 10 x 20 values
    initial_uniform_values = np.random.uniform(0, 1, (pop_size, chromosome_length))
    # round off to either 0 or 1
    initial_binary_population_boolean = initial_uniform_values >= 0.5
    initial_binary_population = initial_binary_population_boolean.astype(int)

    return initial_binary_population

In [7]:
initial_binary_population = init_population(10, 20)
print(initial_binary_population)

[[0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 1 1 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0]
 [0 1 1 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0 1 1]
 [0 1 1 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1 0 0]
 [0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0]
 [1 0 1 0 0 0 1 0 1 0 0 1 1 0 1 1 1 0 0 1]
 [1 1 1 1 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0]
 [0 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1]
 [0 1 1 0 1 0 0 0 1 1 1 0 0 0 0 0 1 1 0 0]
 [1 1 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 1]]


In [8]:
def bin_to_dec(bin_list):
    """
    bin_to_dec(bin_list)
    function that takes in a 2D list of binary values and converts 
    each list to decimal (base 2 format). The function treats each 
    item in the list as if they were a continuous binary number by 
    joining them together and does the conversion to decimal format
    bin_list: the 2D list
    Returns: a list of the converted base 2 numbers
    """
    #empty list that will contain the phenotyoes(base 2 values of the binary numbers)
    phenotypes = []

    # iteratively loop through the list and stringify the binary values then convert to base 2 int
    for i in bin_list:
        bin_str = ''
        for j in i:
            bin_str += str(j)
        phenotypes.append(int(bin_str, 2))

    return phenotypes

In [10]:
def solar_energy_fitness_calc(bin_list, min, max):
    """
    solar_energy_fitness_calc(bin_list, min, max)
    function that calculates the fitness values of each binary 
    number being evaluated in the genetic algorithm. The binary 
    number list is passed in as a 2D array whose values are lists 
    of binary values. The function treats each item in the list 
    as if they were a continuous binary number by joining them 
    together and does the conversion to decimal format before 
    calculating the fitness value
    bin_list: the 2D list
    min: the lower bound of the solar energy test problem
    max: the upper bound of the solar energy test problem
    Returns: a list of the fitness values of the binary values
    """
    # empty list that will store the fitness values of the binary values
    fitness_vals = []

    # iteratively convert the binary values to base 2 which you pass to the fitness function
    for index, i in enumerate(bin_to_dec(bin_list)):
        fitness_vals.append(min + (((max - min) * i) / 2**len(bin_list[index]) - 1))

    return fitness_vals

In [11]:
def crossover(bin_parents_list, type='single-point'):
    """
    crossover(bin_parents_list, type='single-point')
    function that performs a crossover of 2 parents in the population
    and returns 2 children
    The type of crossover is dependent on the argument passed in as an
    argument. Crossover is done by breaking down the chromosome at randomly
    selected points and concatenating with the other parents genes from that
    position. This is done without changing the genes of the parents since new
    children chromosomes are created.
    bin_parents_list: a list of 2 parents that need a crossover done on them
    type(optional): the type of crossover desired to be done. The default type 
    is single point
    returns: a list of 2 children that result from the crossover
    """
    bin_children_list = []

    if type == 'single-point':
        # if the type is set to single-point, perform a single-point crossover
        crossover_point = randint(1, len(bin_parents_list[0])-1)
        '''
        slice the list from beginning to randomly selected index,
        append slice of second list from that point to the end to the first.
        perform the inverse on the second parent
        '''
        bin_children_list.append([*bin_parents_list[0][:crossover_point],\
                                  *bin_parents_list[1][crossover_point:]])
        bin_children_list.append([*bin_parents_list[1][:crossover_point],\
                                  *bin_parents_list[0][crossover_point:]])
    elif type == 'two-point':
        # if the type is set to two-point, perform a two-point crossover
        crossover_point = randint(low=1, high=len(bin_parents_list[0])-1, size=2)
        '''
        slice the list from beginning to the first randomly selected index, insert 
        part of the second part from that index up until the second randomly selected 
        index and add the last part of the forst index from that point to the end
        perform the inverse on the second parent
        '''
        bin_children_list.append([*bin_parents_list[0][:crossover_point[0]],\
                                  *bin_parents_list[1][crossover_point[0]:crossover_point[1]],\
                                  *bin_parents_list[0][crossover_point[1]:]])
        bin_children_list.append([*bin_parents_list[1][:crossover_point[0]],\
                                  *bin_parents_list[0][crossover_point[0]:crossover_point[1]],\
                                  *bin_parents_list[1][crossover_point[1]:]])
    elif type == 'uniform':
        # if the type is set to uniform, perform uniform crossover
        child1 = []
        child2 = []
        # randomly and uniformly swap bits between the two parents
        for i in range(len(bin_parents_list[0])):
            crossover_point = randint(0, 2)
            if crossover_point == 1:
                child1.append(bin_parents_list[1][i])
                child2.append(bin_parents_list[0][i])
            else:
                child1.append(bin_parents_list[0][i])
                child2.append(bin_parents_list[1][i])
        bin_children_list.append(child1)
        bin_children_list.append(child2)
                
    return np.array(bin_children_list)

In [12]:
print(initial_binary_population[0:2])
print(crossover(initial_binary_population[0:2]))

[[0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 1 1 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0]]
[[0 0 1 0 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 1 0 0 1 1 1 0 1 0 1 1 0]]


In [13]:
print(initial_binary_population[0:2])
print(crossover(initial_binary_population[0:2], type='two-point'))

[[0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 1 1 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0]]
[[0 0 1 0 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 1 0 0 1 1 1 0 1 0 1 1 0]]


In [14]:
print(initial_binary_population[0:2])
print(crossover(initial_binary_population[0:2], type='uniform'))

[[0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 1 1 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0]]
[[0 1 1 0 0 0 0 0 1 0 1 1 0 1 0 1 0 1 1 0]
 [1 0 1 1 0 0 0 0 1 1 0 1 1 1 0 0 1 0 1 0]]


In [15]:
def mutate(bin_list, mutation_rate=0.1, mutation_ratio=0.1):
    """
    mutate(bin_list, mutation_rate=0.1, mutation_ratio=0.1)
    function that performs mutation of random individuals in
    a binary population. The number of individuals on which mutation
    will take place and the number of bits in an individual affected by
    the mutation are determined by the mutation rate and mutation ratio
    respectively
    bin_list: the binary population on in which mutation will is performed
    mutation_rate(optional): The percentage of the populations being mutated
    mutation_ratio(optional): The percentage of bits in the population being mutated
    returns: the population that has been mutated based in the parameters provided
    """
    '''
    randomly obtain the indices of the individuals being mutated. 
    The number of indices will approximately be based on the percentage 
    provided as the mutation_rate parameter
    '''
    mutation_chromosome_indices = choice(np.array(range(len(bin_list))),\
                                         math.ceil(len(bin_list) * mutation_rate),\
                                         replace=False)

    print("Mutation Chromosome indices:"+str(mutation_chromosome_indices))
    # loop through each individual in the population
    for i in mutation_chromosome_indices:
        '''
        randomly obtain the indices of the bits being mutated. 
        The number of indices will approximately be based on the percentage 
        provided as the mutation_ratio parameter
        '''
        mutation_gene_indices = choice(np.array(range(len(bin_list[i]))),\
                                         int(len(bin_list[i]) * mutation_ratio),\
                                         replace=False)
        # flip the bit
        for j in mutation_gene_indices:
            bin_list[i][j] = abs(bin_list[i][j] - 1)
        
    return np.array(bin_list)

In [16]:
print("Population before mutation:\n"+str(initial_binary_population))
print("------------------------------------------")
print("Population after mutation:\n"+str(mutate(initial_binary_population)))

Population before mutation:
[[0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 1 1 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0]
 [0 1 1 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0 1 1]
 [0 1 1 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1 0 0]
 [0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0]
 [1 0 1 0 0 0 1 0 1 0 0 1 1 0 1 1 1 0 0 1]
 [1 1 1 1 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0]
 [0 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1]
 [0 1 1 0 1 0 0 0 1 1 1 0 0 0 0 0 1 1 0 0]
 [1 1 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 1]]
------------------------------------------
Mutation Chromosome indices:[5]
Population after mutation:
[[0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 1 1 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0]
 [0 1 1 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0 1 1]
 [0 1 1 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1 0 0]
 [0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0]
 [1 0 1 0 1 0 1 0 1 0 0 1 1 0 1 0 1 0 0 1]
 [1 1 1 1 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0]
 [0 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1]
 [0 1 1 0 1 0 0 0 1 1 1 0 0 0 0 0 1 1 0 0]
 [1 1 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 1]]


In [17]:
def solar_energy_cost_function(bin_list, min, max):
    """
    solar_energy_cost_function(bin_list, min, max)
    function that takes in a binary population and calclulates their
    fitness values. This is first done by decoding the value which is
    fed into the cost function. The fitness values are appended to an array of
    fitness values which are then paired with their corresponsing individuals
    and the result is returned by the function
    bin_list: the list containing the binary population
    min: the lower bound value of the cost function
    max: the upper bound value of the cost function
    returns: list of tuples of binary population with corresponding fitness values
    """
    # create empty numpy array that will hold the calculated fitness values
    fitness_vals = []
    
    # decode the binary into values that are values for the fitness function
    decoded_vals = solar_energy_fitness_calc(bin_list, min, max)
    # lop through each decoded value and pass through the cost function to obtain the cost/fitness values
    for i in decoded_vals:
        val = [(204165.5 / (330 - 2 * i)) + (10400 / (i - 20))]
        fitness_vals = np.append(fitness_vals, val)

    pop_fitness_data = pop_fitness_combine(bin_list, fitness_vals)
    
    return pop_fitness_data

In [18]:
def tournament_fitness_selection(pop_fitness_data, min_selection=4, selection_limit=8):
    comparison_pairs = []
    selected_winners = []
    while len(selected_winners) < min_selection:  
        for i in range(len(pop_fitness_data)):
            possible_pair_indices = [j for j in range(len(pop_fitness_data)) if j != i]
            comparison_pairs.append((i, choice(possible_pair_indices)))
        for j in comparison_pairs:
            if pop_fitness_data[j[0]][1] < pop_fitness_data[j[1]][1]:
                selected_winners.append(pop_fitness_data[j[0]][0])
    if len(selected_winners) > selection_limit:
        selected_winners = selected_winners[:selection_limit]

    return selected_winners

In [19]:
def pop_fitness_combine(bin_pop, fitness_vals):
    pop_fitness_data = []

    print("********************************************Performance**********************************")
    for i in range(len(bin_pop)):
        print(f"Chromosome: {bin_pop[i]}--------Score: {fitness_vals[i]}")
        pop_fitness_data.append((bin_pop[i], fitness_vals[i]))
    print("*****************************************************************************************")
    return pop_fitness_data

In [20]:
def crossover_parent_pairing(selection_winners):
    return [selection_winners[i:i+2] for i in range(0, len(selection_winners), 2)]

In [21]:
def create_new_generation(selection_winners):
    parents = crossover_parent_pairing(selection_winners)
    children = []

    for i in parents:
        if len(i) == 2:
            crossover_result = crossover(i)
            for j in crossover_result:
                children.append(j)
    
    return np.array(selection_winners + children)

In [22]:
def discrete_genetic_algorithm(init_bin_population, min_bound, max_bound, generations=20):
    population = init_bin_population
    for i in range(generations):
        print(f"**********************************Generation {i + 1}**********************************")
        pop_fitness_data = solar_energy_cost_function(population, min_bound, max_bound)
        selection_winners = tournament_fitness_selection(pop_fitness_data)
        parent_children = create_new_generation(selection_winners) # This includes parent pairing and crossover
        population = mutate(parent_children)

# Implementation

### Testing the algorithm flow

In [25]:
initial_binary_population = init_population(10, 20)
print(initial_binary_population)

[[0 0 1 0 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0 1 1 1]
 [0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0]
 [0 1 0 1 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 1]
 [0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 1 0 0 0 0 1 1 1 0 0 1]
 [0 1 1 0 0 1 1 1 0 1 0 0 1 0 0 1 0 1 1 1]
 [0 0 0 0 1 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0]
 [0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 1 1 1 0 0]]


In [26]:
pop_fitness_data = solar_energy_cost_function(initial_binary_population, 40, 90)

********************************************Performance**********************************
Chromosome: [0 0 1 0 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 0]--------Score: 1242.1095916542763
Chromosome: [0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0 1 1 1]--------Score: 1334.2660511163233
Chromosome: [0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0]--------Score: 1237.4518759735874
Chromosome: [0 1 0 1 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 1]--------Score: 1226.7492724597719
Chromosome: [0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0]--------Score: 1233.0620072026456
Chromosome: [0 0 0 0 1 0 0 0 1 1 0 0 0 0 1 1 1 0 0 1]--------Score: 1323.469976167109
Chromosome: [0 1 1 0 0 1 1 1 0 1 0 0 1 0 0 1 0 1 1 1]--------Score: 1230.1086319673357
Chromosome: [0 0 0 0 1 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0]--------Score: 1302.3991682573421
Chromosome: [0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0 0 0 0 1]--------Score: 1301.773584476347
Chromosome: [0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 1 1 1 0 0]--------Score: 1296.5387743752024
******************************************

In [27]:
selection_winners = tournament_fitness_selection(pop_fitness_data)
print(selection_winners)

[array([0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]), array([0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1]), array([0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0])]


In [28]:
parents = crossover_parent_pairing(selection_winners)
print(parents)

[[array([0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]), array([0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1])], [array([0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1])], [array([0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0])]]


In [29]:
children = []
for i in parents:
    if len(i) == 2:
        crossover_result = crossover(i)
        for j in crossover_result:
            children.append(j)

print(children)

[array([0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1]), array([0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]), array([0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0])]


In [30]:
parent_children = np.array(selection_winners + children)
print(parent_children)

[[0 0 1 0 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 0]
 [0 1 0 1 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 1]
 [0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0]
 [0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 1 1 1 0 0]
 [0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 0 1 1 1]
 [0 1 0 1 1 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 1 0 1 1 1 1 1 0 1 0 1 1 0 0 0 0 0 1]
 [0 0 0 1 1 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0]]


In [31]:
new_population = mutate(parent_children)
new_population

Mutation Chromosome indices:[4]


array([[0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1],
       [0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0]])

In [32]:
initial_binary_population = init_population(50, 20)

In [33]:
discrete_genetic_algorithm(initial_binary_population, 40, 90)

**********************************Generation 1**********************************
********************************************Performance**********************************
Chromosome: [1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 1 1 0]--------Score: 1386.7985847654886
Chromosome: [1 1 0 0 1 0 1 1 0 1 0 1 1 0 1 1 0 1 0 1]--------Score: 1360.2480131763914
Chromosome: [1 0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 0 0]--------Score: 1254.1312419909918
Chromosome: [0 0 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 1]--------Score: 1262.584259349575
Chromosome: [1 0 0 0 1 0 1 1 1 1 1 0 1 0 1 0 0 0 0 1]--------Score: 1259.0472497870112
Chromosome: [0 1 1 0 1 0 0 1 1 0 0 1 0 0 1 0 1 1 1 0]--------Score: 1231.204210293864
Chromosome: [1 1 1 0 0 0 0 0 1 1 0 0 1 1 1 1 0 0 0 1]--------Score: 1408.8369079186948
Chromosome: [0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1]--------Score: 1245.303536177732
Chromosome: [1 0 0 0 1 1 1 0 0 0 1 0 0 1 1 1 0 1 0 0]--------Score: 1261.5534109503062
Chromosome: [0 0 1 1 0 0 1 1 1 0 1 0 1 0 1 1 1 1 